In [17]:
import pandas as pd
import tensorflow as tf
import sklearn
import scikeras

In [18]:
import time
from scikeras.wrappers import KerasRegressor
from tensorflow.keras import backend as k
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn import metrics

In [19]:
inicio = time.time()

In [20]:
inicio

1726249761.8688905

In [21]:
base = pd.read_csv('autos.csv', encoding = 'ISO-8859-1')

In [22]:
base = base.drop('dateCrawled', axis = 1)
base = base.drop('dateCreated', axis = 1)
base = base.drop('nrOfPictures', axis = 1)
base = base.drop('postalCode', axis = 1)
base = base.drop('lastSeen', axis = 1)
base = base.drop('name', axis = 1)
base = base.drop('seller', axis = 1)
base = base.drop('offerType', axis = 1)

In [23]:
base = base[base.price > 10]
base = base.loc[base.price < 350000]

In [24]:
valores = {
    'vehicleType': 'limousine',
    'gearbox': 'manuell',
    'model': 'golf',
    'fuelType': 'benzin',
    'brand': 'volkswagen',
    'notRepairedDamage': 'nein'
}
base = base.fillna(value = valores)

In [25]:
x = base.iloc[:, 1:12].values
y = base.iloc[:, 0].values

In [26]:
x, y

(array([['test', 'limousine', 1993, ..., 'benzin', 'volkswagen', 'nein'],
        ['test', 'coupe', 2011, ..., 'diesel', 'audi', 'ja'],
        ['test', 'suv', 2004, ..., 'diesel', 'jeep', 'nein'],
        ...,
        ['test', 'bus', 1996, ..., 'diesel', 'volkswagen', 'nein'],
        ['test', 'kombi', 2002, ..., 'diesel', 'volkswagen', 'nein'],
        ['control', 'limousine', 2013, ..., 'benzin', 'bmw', 'nein']],
       dtype=object),
 array([  480, 18300,  9800, ...,  9200,  3400, 28990], dtype=int64))

In [27]:
onehotencoder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0, 1, 3, 5, 8, 9, 10])],remainder='passthrough')
x = onehotencoder.fit_transform(x).toarray()

In [28]:
x.shape

(359291, 316)

In [29]:
def criar_rede():
    k.clear_session()
    regressor = Sequential([
        tf.keras.layers.InputLayer(shape = (316,)),
        tf.keras.layers.Dense(units = 158, activation = 'relu'),
        tf.keras.layers.Dense(units = 158, activation = 'relu'),
        tf.keras.layers.Dense(units = 1, activation = 'linear'),
    ])

    regressor.compile(loss = 'mean_absolute_error', optimizer = 'adam', metrics = ['mean_absolute_error'])
    return regressor

In [30]:
regressor = KerasRegressor(model = criar_rede, epochs = 100, batch_size = 300)

In [31]:
resultados = cross_val_score(estimator = regressor, X = x, y = y,
                             cv = 5, scoring = 'neg_mean_absolute_error')

Epoch 1/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 4404.8628 - mean_absolute_error: 4404.8628
Epoch 2/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3492.4504 - mean_absolute_error: 3492.4504
Epoch 3/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 3236.5166 - mean_absolute_error: 3236.5166
Epoch 4/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2963.2239 - mean_absolute_error: 2963.2239
Epoch 5/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2864.1951 - mean_absolute_error: 2864.1951
Epoch 6/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2815.6448 - mean_absolute_error: 2815.6448
Epoch 7/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2781.2834 - mean_absolute_error: 2781.2834
Epoch 8/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2731.1365 - mean_absolute_error: 2731.1365
Epoch 9/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 2699.1846 - mean_absolute_error: 2699.1846
Epoch 10/100
959/959 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 26

In [32]:
fim = time.time()
fim

1726250653.960114

In [33]:
(fim - inicio) / 60 / 60

0.24780311763286592

In [34]:
abs(resultados)

array([2243.99965622, 2631.64457361, 2231.34220775, 2390.51602352,
       2331.25401752])

In [35]:
abs(resultados.mean())

2365.751295725567

In [37]:
resultados.std()

145.18392047471673